In [ ]:
from datetime import datetime

def log_message(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] {message}")

silver_base_path = "abfss://fintech@fintechdatagdsde.dfs.core.windows.net/silver/"

log_message("Checking Silver layer structure...")

# List silver folder
try:
    folders = mssparkutils.fs.ls(silver_base_path)
    log_message(f"Folders in silver: {[f.name for f in folders]}")
except Exception as e:
    log_message(f"Error: {e}")

# Check Accounts folder contents
try:
    log_message("\nChecking Accounts folder...")
    contents = mssparkutils.fs.ls(f"{silver_base_path}Accounts/")
    log_message(f"Items in Accounts: {len(contents)}")
    for item in contents[:10]:
        log_message(f"  {item.name} - {item.size} bytes")
    
    has_delta = any('_delta_log' in item.name for item in contents)
    log_message(f"\nHas _delta_log: {has_delta}")
    
except Exception as e:
    log_message(f"Error listing Accounts: {e}")

# Try reading as Parquet (not Delta)
try:
    log_message("\nTrying to read as Parquet...")
    df = spark.read.parquet(f"{silver_base_path}Accounts/")
    log_message(f"SUCCESS! Read as Parquet - {df.count()} rows")
    df.printSchema()
except Exception as e:
    log_message(f"Failed to read as Parquet: {e}")

# Try reading as Delta
try:
    log_message("\nTrying to read as Delta...")
    df = spark.read.format("delta").load(f"{silver_base_path}Accounts/")
    log_message(f"SUCCESS! Read as Delta - {df.count()} rows")
except Exception as e:
    log_message(f"Failed to read as Delta: {e}")

StatementMeta(gdssparkpool, 6, 8, Finished, Available, Finished)